First we include some helper functions to interact with OpenAI's API. 

In [1]:
from src.config import client, model
from src.utils import load_config
from src.assistants.analyst.literature import literature_search
import json
import os

def create_message_and_run_analyst_module(assistant, instructions, user_prompt, thread = None, save_tool_request_path = None):
    '''
    This function is a slightly simplified version of how the analyst module would be in WildfireGPT.
    In this experiment, we only seek to understand how the changes in user profile affect the retrieved literature and the recommendations.
    '''
    if thread is None:
        thread = client.beta.threads.create()
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_prompt
    )
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=instructions,
        parallel_tool_calls=False
    )

    tool_outputs = []

    if run.required_action is not None:
        # Loop through each tool in the required action section
        for tool in run.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "literature_search":
                function_args = json.loads(tool.function.arguments)
                print(function_args)
                if save_tool_request_path is not None:
                    with open(save_tool_request_path, "w") as f:
                        f.write(json.dumps(function_args))
                function_response = literature_search(**function_args)
                path = 'src/assistants/analyst/appendix/literature.md'
                with open(path, "r") as f:
                    appendix = f.read()
                    function_response += appendix
                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": function_response
                })

        if len(tool_outputs) > 0:
            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs,
            )

    return thread
    

def generate_plan(profile):
    '''
    This function is a slightly simplified version of how the planning module would be in WildfireGPT.
    In this experiment, we only seek to understand how the changes in user profile affect the retrieved literature and the recommendations.
    '''
    plan_config = load_config("src/assistants/plan/config.yml")
    plan_instruction = f"{plan_config['instructions']}\n{plan_config['available_datasets']}\n{plan_config['example']}\nHere is the information about your client: {profile}"
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": plan_instruction},
            {"role": "user", "content": "Please explain your plan to me. Simply output the plan and nothing else."},
        ]
    )
    return completion.choices[0].message.content


# There are five different user profiles that we will be using in this ablation experiment. 


template_profile = '''- **Profession:** {profession} in Virginia.
- **Concern:** Managing the forest, keeping it healthy, while {concern}, and protecting properties from potential wildfires.
- **Location:** Near Covington, VA with Latitude 37.7935 and Longitude -79.9939.
- **Time:** Recommendations to be implemented within the next 5 to 10 years.
- **Scope:** Management of the forest and properties to maximize {scope}, and protect against potential wildfires.
'''

profiles = {
    "homeowner": {
        "profession": "Homeowner",
        "concern": "maximizing marketable species",
        "scope": "health and marketable species",
    },
    "civil_engineer": 
    {
        "profession": "Civil Engineer",
        "concern": "ensuring structural and infrastructural resilience",
        "scope": "drainage efficiency and slope stability",
    },
    "ecologist": 
    {
        "profession": "Ecologist",
        "concern": "maintaining biodiversity and ecosystem services",
        "scope": "ecological resilience and habitat connectivity",
    },
    "emergency_manager": 
    {
        "profession": "Emergency Manager",
        "concern": "establishing defendable space and evacuation corridors",
        "scope": "emergency access and response capabilities",
    },
    "power_grid_manager": 
    {
        "profession": "Power Grid Manager",
        "concern": "maintaining transmission line clearance and grid resilience",
        "scope": "power distribution reliability and access",
    },
}

/Users/yangxinyuxie/Library/Caches/pypoetry/virtualenvs/src-KW4cdEL7-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In the code block below, we print the original instruction to the Analyst Module from the case study `case_studies/Private Property Protection`. 

In the ablation study below, we modify the user profile slightly to see how the model's predictions change. Specifically, we try to observe changes in 
- the plan proposed by the planning module with the updated user profile
- the content of the literature review generated by the analyst module with the updated user profile and plan
- the content of the recommendation generated by the analyst module with the updated user profile, plan, and literature review

There are five different user profiles that we will be using in this ablation experiment. They are: homeowner, civil_engineer, ecologist, emergency_manager, and firefighter. You can see the minimal level of changes made to the user profile in the code block above.

We will save everything under the folder `ablation_study` in the current working directory for reference and comparison.

In [2]:
thread_messages = client.beta.threads.messages.list("thread_KKbnKKvkPOFivz8r5ZAYPUCn")
assistant = client.beta.assistants.retrieve(thread_messages.data[0].assistant_id)
print(assistant.instructions)

As an expert consultant specializing in wildfire risks, your role is to assist your client with various aspects of wildfire and climate change understanding and mitigation. Effectively engage with your client in order to address their concerns.
Here is the information about your client: 
- **Profession:** Homeowner in Virginia.
- **Concern:** Managing the forest, keeping it healthy, while maximizing marketable species, and protecting properties from potential wildfires.
- **Location:** Near Covington, VA with Latitude 37.7935 and Longitude -79.9939.
- **Time:** Recommendations to be implemented within the next 5 to 10 years.
- **Scope:** Management of the forest and properties to maximize health and marketable species, and protect against potential wildfires.
- **Vegetation and Forest Composition:** Conifer and broadleaf species, with a focus on maintaining oak and cherry trees.
- **Fire History and Management Practices:** Uncertain about recent wildfires; no specific past management p

We use a straightforward prompt for recommendations to show the effect of the user profile on the model's predictions.
> Develop recommendations to enhance forest resilience against wildfires.

We will first run the model with this prompt only and then with the different versions of the user profile and prompt to see how the model's predictions change.

In [ ]:
def get_response_and_save(recommendation_user_prompt, save_path = None):
    if os.path.exists(save_path):
        # read the response from the file
        with open(save_path, 'r') as f:
            response = f.read()
    else:
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": recommendation_user_prompt},
            ]
        )
        response = completion.choices[0].message.content
    
    print(response)
    with open(save_path, 'w') as f:
        f.write(response)

literature_search_user_prompt = "Let's review some relevant literature."

recommendation_user_prompt = "Develop recommendations to enhance forest resilience against wildfires."

get_response_and_save(recommendation_user_prompt, 'ablation_study/recommendation_base.md')

Enhancing forest resilience against wildfires is crucial for preserving biodiversity, protecting infrastructure, and mitigating climate change impacts. Here is a multi-faceted approach to increasing forest resilience:

1. **Fuel Management**: 
   - **Thinning and Pruning**: Reduce forest density through thinning and pruning, which can lower fire intensity by removing small trees and underbrush that serve as ladder fuels. 
   - **Controlled Burns**: Implement controlled (prescribed) burns to reduce available fuels. This needs to be carefully managed to minimize ecological damage.
   - **Remove Dead Wood**: Clear dead wood and other debris that can intensify fires.

2. **Forest Planning and Layout**:
   - **Strategic Planting**: Use fire-resilient tree species in reforestation efforts and diversify age classes and species to create a more heterogeneous landscape which is less susceptible to widespread fires.
   - **Buffer Zones**: Develop buffer zones using less flammable species around 

Now, we add the location and time from the user profile to the prompt to see how the model's predictions change.

In [ ]:
recommendation_user_prompt_with_location_and_time = f'''- **Location:** Near Covington, VA with Latitude 37.7935 and Longitude -79.9939.
- **Time:** Recommendations to be implemented within the next 5 to 10 years.
{recommendation_user_prompt}'''

get_response_and_save(recommendation_user_prompt_with_location_and_time, 'ablation_study/recommendation_location_time.md')

Given the location near Covington, VA, and the current conditions conducive to wildfires, future forest management strategies must integrate robust, long-term approaches to enhancing forest resilience. Using a timeframe of 5 to 10 years, here are several recommendations specifically tailored for the outlined region:

1. **Fire-Adapted Ecosystem Promotion:**
   - Develop and promote fire-adapted ecosystems by incorporating native species that are more resistant to fire. This includes the use of oak and hickory trees which tend to be more resilient against fires compared to other species like pines.

2. **Controlled Burning:**
   - Implement regular controlled or prescribed burns. These managed fires eliminate underbrush, dead trees, and leaf litter that contribute to wildfire intensity. This practice also helps certain species to germinate and thrive, thus maintaining the ecological balance.

3. **Strategic Firebreaks:**
   - Build and maintain natural and artificial firebreaks such as 

In [5]:
# Now set the identify of the user
user_identity = "power_grid_manager"

def generate_profile(profile):
    profile_text = template_profile.format(**profile)
    return profile_text

profile = generate_profile(profiles[user_identity])
print(profile)

- **Profession:** Power Grid Manager in Virginia.
- **Concern:** Managing the forest, keeping it healthy, while maintaining transmission line clearance and grid resilience, and protecting properties from potential wildfires.
- **Location:** Near Covington, VA with Latitude 37.7935 and Longitude -79.9939.
- **Time:** Recommendations to be implemented within the next 5 to 10 years.
- **Scope:** Management of the forest and properties to maximize power distribution reliability and access, and protect against potential wildfires.



In [ ]:
recommendation_user_prompt_with_location_and_time = f'''{profile}\n\n
{recommendation_user_prompt}'''

get_response_and_save(recommendation_user_prompt_with_location_and_time, 'ablation_study/recommendation_grid_manager.md')

To optimize forest management, enhance grid reliability, and shield against wildfires near Covington, VA, a multifaceted approach that spans the next 5 to 10 years is recommended. Here are strategic steps tailored to your profession as a Power Grid Manager:

### 1. **Vegetation Management and Clearance Regulations**
   - **Regular Maintenance:** Establish routine schedules for pruning and removing trees that pose a risk to power lines. Utilize drones or satellite imagery to monitor vegetation growth for timely intervention.
   - **Right-of-Way (ROW) Expansion:** Consider expanding the ROW clearance around transmission lines beyond standard regulations to create a buffer zone that reduces wildfire risks and ensures operational safety.
   - **Integrated Vegetation Management (IVM):** Implement an IVM program that combines mechanical, biological, and chemical methods to manage vegetation. This approach balances ecological health and electrical safety efficiently.

### 2. **Grid Infrastruc

In [8]:
import os
if not os.path.exists(f"ablation_study/{user_identity}"):
    os.makedirs(f"ablation_study/{user_identity}")

# load the plan if it exists
if os.path.exists(f"ablation_study/{user_identity}/plan.md"):
    with open(f"ablation_study/{user_identity}/plan.md", "r") as f:
        plan = f.read()
else:
    plan = generate_plan(profile)
    with open(f"ablation_study/{user_identity}/plan.md", "w") as f:
        f.write(plan)

print(plan)


**Step 1**: Analyze the Recent fire incident data to understand recent trends and specific areas of higher fire incidence near Covington, VA. This will help in assessing the most vulnerable spots along the transmission lines and around residential areas that need urgent attention for vegetation management and infrastructure protection.

**Step 2**: Use the Long term fire history records to gather insights into historical wildfire patterns in the area. This can offer a broader context of fire behavior over centuries, helping to predict potential future hotspots and periods of higher risk, which is crucial for long-term planning and resilience strategies.

**Step 3**: Conduct a literature search focusing on effective strategies for vegetation management, forest health maintenance, and wildfire risk mitigation around power grids. This will include exploring practices such as controlled burns, vegetation clearance standards, and innovations in infrastructure design that enhance resilience 

In [9]:
analyst_instruction = "Here is the information about your client:\n\n{profile}\n\nHere is your overall plan to assist your client: {plan}\n\nHere is your client's last message: {user_prompt}\n\nHere is your plan for this step: {this_step}\n\nIf you give any recommendations, please provide the reasoning behind them and suggest the client to ask for supporting scientific evidence."

this_step = {
    'literature': "Respond to the client's questions. Analyze literature. `literature_search`", 
    'recommendation': "Respond to the client's questions. Develop recommendations. `no tool needed`"
}

In [11]:
if not os.path.exists(f"ablation_study/{user_identity}/literature_search_query.txt"):

    instrucion = analyst_instruction.format(profile=profile, plan=plan, user_prompt=literature_search_user_prompt, this_step=this_step['literature'])

    thread = create_message_and_run_analyst_module(assistant, instrucion, literature_search_user_prompt, save_tool_request_path = f"ablation_study/{user_identity}/literature_search_query.txt")

    # save the thread id to a file
    with open(f"ablation_study/{user_identity}/thread_id.txt", "w") as f:
        f.write(thread.id)
else:
    with open(f"ablation_study/{user_identity}/literature_search_query.txt", "r") as f:
        query = json.loads(f.read())
    print(query)
    with open(f"ablation_study/{user_identity}/thread_id.txt", "r") as f:
        thread_id = f.read()
    thread = client.beta.threads.retrieve(thread_id)

{'query': 'Effective strategies for vegetation management, forest health maintenance, and wildfire risk mitigation around power grids near Covington, VA'}


In [12]:
# Please don't rush to run this cell. The cell above can have latency as we wait for the response from the OpenAI Assistant API.
# It is a good idea to wait for half to one minutes before running this cell.

# get last message from the thread

if not os.path.exists(f"ablation_study/{user_identity}/literature_search_result.md"):
    thread_messages = client.beta.threads.messages.list(thread.id)

    response = thread_messages.data[0].content[0].text.value

    with open(f"ablation_study/{user_identity}/literature_search_result.md", "w") as f:
        f.write(response)
else:
    with open(f"ablation_study/{user_identity}/literature_search_result.md", "r") as f:
        response = f.read()
print(response)

In the face of increasing wildfire threats, particularly near critical infrastructure like power grids, the literature presents several comprehensive strategies for vegetation management and risk mitigation to enhance forest health and safety. For instance, Hvenegaard (2014) emphasizes the importance of long-term monitoring and data collection at fuel treatment sites to effectively manage vegetation and reduce wildfire risks. This approach is essential for areas like Covington, VA, where maintaining transmission line clearance and grid resilience is critical (Hvenegaard, 2014).

Adding to the discussion, Dale (2006) critiques the current wildfire management policies, which mainly focus on fire suppression, leading to denser vegetation and more intense fires. Dale proposes the adoption of Wildland Fire Use (WFU), which leverages natural fires to meet specific resource management objectives, thus improving forest health and reducing management costs over time. This method could be partic

In [ ]:
if os.path.exists(f"ablation_study/{user_identity}/recommendation.md"):
    with open(f"ablation_study/{user_identity}/recommendation.md", "r") as f:
        response = f.read()
else:
    instrucion = analyst_instruction.format(profile=profile, plan=plan, user_prompt=recommendation_user_prompt, this_step=this_step['recommendation'])
    thread = create_message_and_run_analyst_module(assistant, instrucion, recommendation_user_prompt, thread)

In [ ]:
if not os.path.exists(f"ablation_study/{user_identity}/recommendation.md"):
    # get last message from the thread
    thread_messages = client.beta.threads.messages.list(thread.id)
    response = thread_messages.data[0].content[0].text.value

    with open(f"ablation_study/{user_identity}/recommendation.md", "w") as f:
        f.write(response)
        
print(response)

To enhance forest resilience against wildfires near Covington, VA, especially around critical infrastructure like power grids, I recommend the following strategies based on the reviewed literature and best practices in the field:

1. **Implement Long-term Monitoring and Data Collection:** Establish a continuous monitoring program that tracks vegetation health and wildfire risks in areas surrounding power grids. This should include periodic assessments of forest density, species composition, and fuel accumulation. Monitoring helps in making informed decisions and adapting strategies in response to dynamic environmental conditions (Hvenegaard, 2014).

2. **Adopt an Integrated Fire Management Strategy:** Utilize a mix of suppression and managed wildfire approaches. Where conditions allow, use Wildland Fire Use (WFU) strategies to let naturally ignited fires burn under controlled conditions. This approach helps reduce fuel loads naturally, enhances forest health, and maintains ecological b